In [1]:
import requests
import re, os 
import cv2
from pytube import YouTube
# pytubeはよく壊れるので,壊れてたらgithubのissueを確認する
from bs4 import BeautifulSoup as bs
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

print("[COMPLETED]")

[COMPLETED]


In [2]:
def scrape_videos(query, path, n):
    url ='https://www.youtube.com/results?search_query='+query
    r = requests.get(url)
    page = r.text
    soup=bs(page,'html.parser')
    res=soup.find_all('a',{'href': re.compile(r'watch')})
    i = 0
    for l in res:
        link = "https://www.youtube.com"+l.get("href")
        myVideo = YouTube(link)
        print(link)
        myVideo.streams.first().download(output_path=path, filename='ヒカキン')
        i += 1 
        #print("[INFO] Downloaded {0} for {1}".format(myVideo,title, query))
        print("[COMPLETED3]Downloaded")
        if i >= n:
            break
            
def tsne(x, n):
    X_embedded = TSNE(n_components=n).fit_transform(x)
    return X_embedded

print("[COMPLETED2]")

[COMPLETED2]


# Cut videos into scenes

In [3]:
import subprocess as sp

name_list = open("pyFiles/name_list.txt", "r").read().splitlines()
for line in name_list:
    name, yt_url = line.split(" ")
    path = "./data/videos/{0}/".format(name)
    if not os.path.exists(path):
        os.makedirs(path)
    scrape_videos(name, path, 0)
    
    video_list = os.listdir("./data/videos/{0}/".format(name))
    for fname in video_list:
        print("[INFO] Starting scene detection on {0}".format(fname))
        
        command = ([
            "scenedetect",
            "-i",
            "./data/videos/{0}/{1}".format(name, fname),
            "-o",
            "./data/videos/{0}/".format(name),
            "detect-content",
            "-t",
            "27",
            "split-video",
        ])
        '''proc = sp.run(command, stdout=sp.PIPE, stderr=sp.PIPE)
        if proc.returncode==0:
            print(proc.stdout.decode("utf8"))
        else:
            print(proc.stderr.decode("utf8"))
        os.remove("./data/videos/{0}/{1}".format(name, fname))'''

    print("[COMPLETED] FINISHED Downloading Videos for {0}".format(name))


https://www.youtube.com/watch?v=m5oMWgiN9zY
[COMPLETED3]Downloaded
[INFO] Starting scene detection on みなさんにお願いがあります。.mp4
[INFO] Starting scene detection on 【YouTuber必見】歯医者でも動画編集できる方法を発明しました。.mp4
[INFO] Starting scene detection on ヒカキン.mp4
[COMPLETED] FINISHED Downloading Videos for はじめしゃちょー


# cut roi's in each video

In [5]:
#!pip install dlib
#!pip install face_recognition
#!pip install imutils
#!pip install google_images_download
#!pip install wheel
#!pip install scikit-learn

print("[OK]")
import dlib
from pyFiles.create_image_dataset import *

def warp_face2(ROI, landmarks):
        # Output image
    h = w = 224
    img = np.zeros((h, w, 3), np.float32())

    average_landmarks = np.load("./data/average_landmarks.npy")
    peripheral_points = [
        [0, 0],
        [0, w / 2],
        [0, w - 1],
        [h / 2, 0],
        [h / 2, w - 1],
        [h - 1, 0],
        [h - 1, w / 2],
        [h - 1, w - 1],
    ]
    landmarks = np.append(landmarks, peripheral_points, axis=0)


    # Warp input images to average image landmarks
    dt = Delaunay(average_landmarks).simplices

    # Transform triangles one by ones
    for j in range(0, len(dt)):
        tin = []
        tout = []

        for k in range(0, 3):
            pIn = landmarks[dt[j][k]]
            pIn = constrainPoint(pIn, w, h)

            pOut = average_landmarks[dt[j][k]]
            pOut = constrainPoint(pOut, w, h)

            tin.append(pIn)
            tout.append(pOut)

        warpTriangle(ROI, img, tin, tout)

    return img.astype('uint8')
    
    print("[OK1]")

def check_face(img, name, method):
    face_landmarks = face_recognition.face_landmarks(img, model='large')
    tri_list, pt_list, delaunay_face = get_delaunay(face_landmarks, img)
    ROI, landmarks = tilt_crop_scale(pt_list, img)
    if len(ROI) == 0:
        return -1
    else:
        img = warp_face2(ROI[0], landmarks[0])
        enc = face_recognition.face_encodings(img)
    
        if len(enc) == 0:
            return -2
    
        elif method == 'OCSVM':
            t = clf_dict[name].predict(enc)
            return t
        else:
            return -3

    #if method == 'distance':
        #dist = np.linalg.norm(enc_ave[name] - enc)#enc_ave
        #return dist

weights = './data/mmod_human_face_detector.dat'
face_detector = dlib.cnn_face_detection_model_v1(weights)
face_detector = dlib.get_frontal_face_detector()
print("[COMPLEATED READ]")

name_list = open("./pyFiles/name_list.txt", "r").read().splitlines()
for line in name_list:
    name, yt_url = line.split(" ")
    video_list = os.listdir("./data/videos/{0}/".format(name))

    for fname in video_list:
        input_movie = cv2.VideoCapture("./data/videos/{0}/{1}".format(name, fname))
        length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
        codec = int(input_movie.get(cv2.CAP_PROP_FOURCC))
        fps = int(input_movie.get(cv2.CAP_PROP_FPS))

        frame_width = int(input_movie.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(input_movie.get(cv2.CAP_PROP_FRAME_HEIGHT))
            
        if not os.path.exists("./data/silent_vids/{0}".format(name)):
            os.makedirs("./data/silent_vids/{0}".format(name))
        writer = cv2.VideoWriter("./data/silent_vids/{0}/{1}".format(name, fname),  cv2.VideoWriter_fourcc(*'XVID'), fps, (224,224))

        frame_number=0
        conf_list = [0.5]*(length+1)
        box_list = [None]*(length+1)
        frame_list = [None]*(length+1)
        print(length)
        while True:
            # Grab a single frame of video
            ret, frame = input_movie.read()
            frame_number += 1

            # Quit when the input video file ends
            if not ret:
                break

            # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
            rgb_frame = frame[:, :, ::-1]
            frame_list[frame_number] = rgb_frame

            # Find all the faces and face encodings in the current frame of video
            faces, scores, _ = face_detector.run(rgb_frame, 1, -0.5)
            for i, (face,score) in enumerate(zip(faces, scores)):
                left = face.left()
                top = face.top()
                right = face.right()
                bottom = face.bottom()
                #print(i)
                
                roi = rgb_frame[top:bottom, left:right]
                t = int(check_face(roi, name, 'distance'))#int付け加えた
                if 0 < t and t < 0.5:
                    if box_list[frame_number] == None or t < conf_list[frame_number]:
                        conf_list[frame_number] = t
                        box_list[frame_number] = face
                    else:
                        continue
                print(t)
                    
        print("[OK]")
            
        for i, box in enumerate(box_list):
            if i == 0:
                continue
            elif box == None:
                box_list[i] = box_list[i - 1]
                
        for i, box in enumerate(box_list):
            if box == None:
                roi = np.zeros((224,224,3), np.uint8)
                continue
            else:
                face = box_list[i] 
                rgb_frame = frame_list[i]
                left = face.left()
                top = face.top()
                right = face.right()
                bottom = face.bottom()
                roi = rgb_frame[top:bottom, left:right]
                roi = cv2.resize(roi, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)[:, :, ::-1]
            writer.write(roi)
        writer.release()    
        
        print("[ALL COMPLEATED]")

[OK]
[COMPLEATED READ]
9715
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


FileNotFoundError: [Errno 2] No such file or directory: './data/average_landmarks.npy'